In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
!ls

MANIFEST.in                           gceline_frontend_streamlit.ipynb
Makefile                              notebooks
README.md                             raw_data
WorkforceSentimentMonitoring          requirements.txt
WorkforceSentimentMonitoring.egg-info scripts
app.py                                setup.py
build                                 test_data
dist                                  tests


In [3]:
cd ..

/Users/yunxizzy/code/theresa-sporn/WorkforceSentimentMonitoring


In [23]:
from WorkforceSentimentMonitoring.data import get_data, merge, holdout, encode_target, \
drop_wrong_language, get_prepaired_data

In [24]:
from WorkforceSentimentMonitoring.feature_engineering import SCORE_COLS, FEATURE_COLS,

In [74]:
from WorkforceSentimentMonitoring.encoders import Preprocessor, FeatureEngineer, CustomMinMaxScaler, PredictionFeaturesExtractor

# Load Data

In [40]:
X_train, X_test, y_train, y_test = get_prepaired_data()

Reading data...
Merging data into a single DataFrame...
Identifying entries in other languages...
Drop 434 entries? y / [n]
y
Dropping 434 entries...
Process completed.
Splitting train and test...
Encoding targets...
Done!


In [94]:
display(X_train.head())
display(y_train.head())

,summary,positives,negatives,advice_to_mgmt,review
0,A once great company that has lost its way in ...,Some smart people work at Microsoft Physical w...,Teamwork destroying Stack Ranking Rewards for ...,Get rid of Ballmer and Turner Get rid of Stack...,A once great company that has lost its way in ...
1,GSM,"travel, influence, coworkers, interesting prod...","politics, night hours, advancement opportunities",NaN,"GSM travel, influence, coworkers, interesting ..."
2,My Opinion On Amazon,. Office space Ambiance is good . Compensation...,. Worst work life balance . You have to take c...,. Management should take employee feedback ser...,My Opinion On Amazon . Office space Ambiance i...
3,Still one of the most exciting companies to wo...,breadth of technologies and markets one can wo...,the curve/stack ranking induce some poor behav...,stop bringing decisions to the top. create sep...,Still one of the most exciting companies to wo...
4,Contractors have minimal access to corporate k...,Fabulous talent with honest deliverable within...,"Some slow-to-respond, out-of-date information ...",Management cannot prevent people who want to k...,Contractors have minimal access to corporate k...


,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,1,0,1,1,0,0
1,2,2,2,2,2,2
2,0,0,1,2,0,1
3,2,2,2,2,2,1
4,2,1,0,2,1,1


In [95]:
X_train.isnull().sum()

summary              55
positives             0
negatives             0
advice_to_mgmt    15711
review                0
dtype: int64

In [96]:
X_test.isnull().sum()

summary             25
positives            0
negatives            1
advice_to_mgmt    6733
review               0
dtype: int64

In [41]:
import pandas as pd

In [99]:
X = pd.concat([X_train, X_test])

In [100]:
y = pd.concat([y_train, y_test])

In [102]:
display(X.head())
display(y.head())

,summary,positives,negatives,advice_to_mgmt,review
0,A once great company that has lost its way in ...,Some smart people work at Microsoft Physical w...,Teamwork destroying Stack Ranking Rewards for ...,Get rid of Ballmer and Turner Get rid of Stack...,A once great company that has lost its way in ...
1,GSM,"travel, influence, coworkers, interesting prod...","politics, night hours, advancement opportunities",NaN,"GSM travel, influence, coworkers, interesting ..."
2,My Opinion On Amazon,. Office space Ambiance is good . Compensation...,. Worst work life balance . You have to take c...,. Management should take employee feedback ser...,My Opinion On Amazon . Office space Ambiance i...
3,Still one of the most exciting companies to wo...,breadth of technologies and markets one can wo...,the curve/stack ranking induce some poor behav...,stop bringing decisions to the top. create sep...,Still one of the most exciting companies to wo...
4,Contractors have minimal access to corporate k...,Fabulous talent with honest deliverable within...,"Some slow-to-respond, out-of-date information ...",Management cannot prevent people who want to k...,Contractors have minimal access to corporate k...


,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,1,0,1,1,0,0
1,2,2,2,2,2,2
2,0,0,1,2,0,1
3,2,2,2,2,2,1
4,2,1,0,2,1,1


In [108]:
df = pd.concat([X, y], axis=1)

In [110]:
df.dropna(inplace=True)

In [111]:
df

,summary,positives,negatives,advice_to_mgmt,review,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,A once great company that has lost its way in ...,Some smart people work at Microsoft Physical w...,Teamwork destroying Stack Ranking Rewards for ...,Get rid of Ballmer and Turner Get rid of Stack...,A once great company that has lost its way in ...,1,0,1,1,0,0
2,My Opinion On Amazon,. Office space Ambiance is good . Compensation...,. Worst work life balance . You have to take c...,. Management should take employee feedback ser...,My Opinion On Amazon . Office space Ambiance i...,0,0,1,2,0,1
3,Still one of the most exciting companies to wo...,breadth of technologies and markets one can wo...,the curve/stack ranking induce some poor behav...,stop bringing decisions to the top. create sep...,Still one of the most exciting companies to wo...,2,2,2,2,2,1
4,Contractors have minimal access to corporate k...,Fabulous talent with honest deliverable within...,"Some slow-to-respond, out-of-date information ...",Management cannot prevent people who want to k...,Contractors have minimal access to corporate k...,2,1,0,2,1,1
5,Good Company and environment,-Good working environment -HRs are helpful,"-No metric management. -Sometimes, even if you...",Metrics should be managed in a better way.,Good Company and environment -Good working env...,2,2,1,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...
15705,Working for amazon has been amazing. You have ...,"Working with amazing management is a plus, I l...",As I became a problem solver I thought I'd get...,Just look into problem solving and just give t...,Working for amazon has been amazing. You have ...,2,2,2,2,2,2
15706,SDE,Great learning experience. You learn multiple ...,Too much work. Sometimes it's just too much wo...,Talk to TPM instead of your manager since most...,SDE Great learning experience. You learn multi...,0,2,2,0,2,1
15710,Lost its way,Work life balance is best thing,lack of value adding to company,Start to think again about monetization.,Lost its way Work life balance is best thing l...,2,0,2,2,0,0
15712,Work hard on a huge diversity of products,It's like a bunch of companies under one roof,The review system is hugely unfair,Scrap the review system,Work hard on a huge diversity of products It's...,1,2,2,2,0,0


In [115]:
df.to_csv('raw_data/raw_df.csv', encoding='utf-8', index=False)

## Group in 2 classes

In [129]:
# remap scores into 2 class targets (leave score 3 as '3')
'''def remap(data, dict_labels):
    for key, values in dict_labels.items():
        data.replace({key: {1: '0', 2: '0', 3: '1', 4: '2', 5: '2'}}, inplace=True)
    return data'''

In [114]:
#small_target.apply(lambda row: remap(small_target, small_target), axis=1)

I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping c

I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping 

I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balanc

I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping ove

I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balanc

...

I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping ove

DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapp

I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-valu

I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping ove

0            work-balance culture-values career-opport...
1            work-balance culture-values career-opport...
2            work-balance culture-values career-opport...
3            work-balance culture-values career-opport...
4            work-balance culture-values career-opport...
                              ...                        
9995         work-balance culture-values career-opport...
9996         work-balance culture-values career-opport...
9997         work-balance culture-values career-opport...
9998         work-balance culture-values career-opport...
9999         work-balance culture-values career-opport...
Length: 10000, dtype: object

In [125]:
#small_target = small_target.astype('uint8')

In [ ]:
#small_target

# Prepare 2-class target

In [34]:
# transform scores into 2 classes: 0 => negative; 1 => positive
#target.apply(lambda row: remap(target, target), axis=1)

In [64]:
X_train_copy = X_train.copy()
y_train_copy = y_train.copy()
X_test_copy = X_test.copy()
y_test_copy = y_test.copy()

In [89]:
X_train_copy

,summary,positives,negatives,advice_to_mgmt,review,summary_length,positives_length,negatives_length,advice_to_mgmt_length,review_length,subjectivity_summary,polarity_summary,subjectivity_positives,polarity_positives,subjectivity_negatives,polarity_negatives,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review
0,a once great company that ha lost it way in a ...,some smart people work at microsoft physical w...,teamwork destroying stack ranking reward for p...,get rid of ballmer and turner get rid of stack...,a once great company that ha lost it way in a ...,0.975000,0.007965,0.022204,0.009716,0.024596,0.675000,0.1000,0.511905,0.338095,0.560417,-0.143750,0.000000,0.000000,0.566850,0.004945
1,gsm,travel influence coworkers interesting product,politics night hour advancement opportunity,NaN,gsm travel influence coworkers interesting ...,0.025000,0.005103,0.003042,0.000000,0.003417,0.000000,0.0000,0.500000,0.500000,0.000000,0.000000,NaN,NaN,0.500000,0.500000
2,my opinion on amazon,office space ambiance is good compensation...,worst work life balance you have to take c...,management should take employee feedback ser...,my opinion on amazon office space ambiance i...,0.166667,0.011325,0.023572,0.031126,0.027821,0.000000,0.0000,0.700000,0.525000,0.615000,-0.237500,0.255556,-0.077778,0.553472,0.056597
3,still one of the most exciting company to work...,breadth of technology and market one can work ...,the curve stack ranking induce some poor behav...,stop bringing decision to the top create sepa...,still one of the most exciting company to work...,0.608333,0.014437,0.013763,0.025189,0.023777,0.613333,0.3200,0.500000,0.000000,0.428571,0.128571,0.666667,0.166667,0.529388,0.159388
4,contractor have minimal access to corporate kn...,fabulous talent with honest deliverable within...,some slow to respond out of date information ...,management cannot prevent people who want to k...,contractor have minimal access to corporate kn...,0.491667,0.023398,0.010646,0.035804,0.027436,0.533333,-0.3000,0.633333,0.271429,0.500000,-0.300000,0.200000,-0.200000,0.548889,-0.026667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36661,challenging pace due to legacy organization,opportunity to move to different team,product development involves large number of t...,NaN,challenging pace due to legacy organization op...,0.358333,0.003734,0.006311,0.000000,0.006883,0.687500,0.1875,0.600000,0.000000,0.714286,0.357143,NaN,NaN,0.680714,0.217857
36662,great coworkers and oppurtunities for advance...,i ve gone from a new grade hire to the tech le...,mountain view centric attitude lead to extreme...,NaN,great coworkers and oppurtunities for advance...,0.416667,0.060610,0.022432,0.000000,0.039420,0.750000,0.8000,0.537879,0.242614,0.551389,0.062803,NaN,NaN,0.552975,0.188669
36663,work kamper,great way to adjust to rv living while earning...,it wa a physical job with demanding overtime s...,keep up the good work,work kamper great way to adjust to rv living w...,0.091667,0.011201,0.003498,0.003958,0.007364,0.000000,0.0000,0.750000,0.800000,0.142857,0.000000,0.600000,0.700000,0.560714,0.575000
36664,world class employer but only if you work at h...,smart colleague and management get to work on...,lack of job security for employee who work in ...,be more thoughtful about hiring decision,world class employer but only if you work at h...,0.475000,0.017922,0.021291,0.007377,0.024355,1.000000,0.0000,0.647619,0.304762,0.138750,-0.096250,0.500000,0.450000,0.449786,0.142929


In [90]:
y_train_copy

,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,1,0,1,1,0,0
1,2,2,2,2,2,2
2,0,0,1,2,0,1
3,2,2,2,2,2,1
4,2,1,0,2,1,1
...,...,...,...,...,...,...
36661,2,2,2,1,2,2
36662,2,2,2,2,2,2
36663,1,2,1,1,0,2
36664,0,2,2,2,2,2


## Prepare Features

In [50]:
from WorkforceSentimentMonitoring.feature_engineering import get_lengths,\
get_linear_regression_cols, get_subjectivity_polarity_columns,\
getPolarity, getSubjectivity, add_multinomial_nb_prediction_feature

In [65]:
X_train_copy

,summary,positives,negatives,advice_to_mgmt,review
0,A once great company that has lost its way in ...,Some smart people work at Microsoft Physical w...,Teamwork destroying Stack Ranking Rewards for ...,Get rid of Ballmer and Turner Get rid of Stack...,A once great company that has lost its way in ...
1,GSM,"travel, influence, coworkers, interesting prod...","politics, night hours, advancement opportunities",NaN,"GSM travel, influence, coworkers, interesting ..."
2,My Opinion On Amazon,. Office space Ambiance is good . Compensation...,. Worst work life balance . You have to take c...,. Management should take employee feedback ser...,My Opinion On Amazon . Office space Ambiance i...
3,Still one of the most exciting companies to wo...,breadth of technologies and markets one can wo...,the curve/stack ranking induce some poor behav...,stop bringing decisions to the top. create sep...,Still one of the most exciting companies to wo...
4,Contractors have minimal access to corporate k...,Fabulous talent with honest deliverable within...,"Some slow-to-respond, out-of-date information ...",Management cannot prevent people who want to k...,Contractors have minimal access to corporate k...
...,...,...,...,...,...
36661,Challenging pace due to legacy organization,Opportunity to move to different teams.,Product development involves large number of t...,NaN,Challenging pace due to legacy organization Op...
36662,"Great coworkers, and oppurtunities for advance...",I've gone from a new grade hire to the tech le...,Mountain View-centric attitude leads to extrem...,NaN,"Great coworkers, and oppurtunities for advance..."
36663,Work Kamper,Great way to adjust to RV living while earning...,It was a physical job with demanding overtime ...,Keep up the good work.,Work Kamper Great way to adjust to RV living w...
36664,World class employer but only if you work at h...,Smart colleagues and management. Get to work o...,Lack of job security for employees who work in...,Be more thoughtful about hiring decisions.,World class employer but only if you work at h...


In [66]:
preprocessor = Preprocessor()

In [67]:
X_train_copy = preprocessor.fit_transform(X_train_copy)

In [68]:
X_train_copy

,summary,positives,negatives,advice_to_mgmt,review
0,a once great company that ha lost it way in a ...,some smart people work at microsoft physical w...,teamwork destroying stack ranking reward for p...,get rid of ballmer and turner get rid of stack...,a once great company that ha lost it way in a ...
1,gsm,travel influence coworkers interesting product,politics night hour advancement opportunity,NaN,gsm travel influence coworkers interesting ...
2,my opinion on amazon,office space ambiance is good compensation...,worst work life balance you have to take c...,management should take employee feedback ser...,my opinion on amazon office space ambiance i...
3,still one of the most exciting company to work...,breadth of technology and market one can work ...,the curve stack ranking induce some poor behav...,stop bringing decision to the top create sepa...,still one of the most exciting company to work...
4,contractor have minimal access to corporate kn...,fabulous talent with honest deliverable within...,some slow to respond out of date information ...,management cannot prevent people who want to k...,contractor have minimal access to corporate kn...
...,...,...,...,...,...
36661,challenging pace due to legacy organization,opportunity to move to different team,product development involves large number of t...,NaN,challenging pace due to legacy organization op...
36662,great coworkers and oppurtunities for advance...,i ve gone from a new grade hire to the tech le...,mountain view centric attitude lead to extreme...,NaN,great coworkers and oppurtunities for advance...
36663,work kamper,great way to adjust to rv living while earning...,it wa a physical job with demanding overtime s...,keep up the good work,work kamper great way to adjust to rv living w...
36664,world class employer but only if you work at h...,smart colleague and management get to work on...,lack of job security for employee who work in ...,be more thoughtful about hiring decision,world class employer but only if you work at h...


In [69]:
feature_engineering = FeatureEngineer()

In [70]:
X_train_copy = feature_engineering.fit_transform(X_train_copy)

100%|██████████| 5/5 [02:30<00:00, 30.07s/it]


In [48]:
X_train_copy

100%|██████████| 5/5 [00:00<00:00, 41.45it/s]


In [71]:
scaler = CustomMinMaxScaler()

In [72]:
X_train_copy_scaled = scaler.fit_transform(X_train_copy)

In [78]:
X_train_copy_scaled

,summary,positives,negatives,advice_to_mgmt,review,summary_length,positives_length,negatives_length,advice_to_mgmt_length,review_length,subjectivity_summary,polarity_summary,subjectivity_positives,polarity_positives,subjectivity_negatives,polarity_negatives,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review
0,a once great company that ha lost it way in a ...,some smart people work at microsoft physical w...,teamwork destroying stack ranking reward for p...,get rid of ballmer and turner get rid of stack...,a once great company that ha lost it way in a ...,0.975000,0.007965,0.022204,0.009716,0.024596,0.675000,0.1000,0.511905,0.338095,0.560417,-0.143750,0.000000,0.000000,0.566850,0.004945
1,gsm,travel influence coworkers interesting product,politics night hour advancement opportunity,NaN,gsm travel influence coworkers interesting ...,0.025000,0.005103,0.003042,0.000000,0.003417,0.000000,0.0000,0.500000,0.500000,0.000000,0.000000,NaN,NaN,0.500000,0.500000
2,my opinion on amazon,office space ambiance is good compensation...,worst work life balance you have to take c...,management should take employee feedback ser...,my opinion on amazon office space ambiance i...,0.166667,0.011325,0.023572,0.031126,0.027821,0.000000,0.0000,0.700000,0.525000,0.615000,-0.237500,0.255556,-0.077778,0.553472,0.056597
3,still one of the most exciting company to work...,breadth of technology and market one can work ...,the curve stack ranking induce some poor behav...,stop bringing decision to the top create sepa...,still one of the most exciting company to work...,0.608333,0.014437,0.013763,0.025189,0.023777,0.613333,0.3200,0.500000,0.000000,0.428571,0.128571,0.666667,0.166667,0.529388,0.159388
4,contractor have minimal access to corporate kn...,fabulous talent with honest deliverable within...,some slow to respond out of date information ...,management cannot prevent people who want to k...,contractor have minimal access to corporate kn...,0.491667,0.023398,0.010646,0.035804,0.027436,0.533333,-0.3000,0.633333,0.271429,0.500000,-0.300000,0.200000,-0.200000,0.548889,-0.026667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36661,challenging pace due to legacy organization,opportunity to move to different team,product development involves large number of t...,NaN,challenging pace due to legacy organization op...,0.358333,0.003734,0.006311,0.000000,0.006883,0.687500,0.1875,0.600000,0.000000,0.714286,0.357143,NaN,NaN,0.680714,0.217857
36662,great coworkers and oppurtunities for advance...,i ve gone from a new grade hire to the tech le...,mountain view centric attitude lead to extreme...,NaN,great coworkers and oppurtunities for advance...,0.416667,0.060610,0.022432,0.000000,0.039420,0.750000,0.8000,0.537879,0.242614,0.551389,0.062803,NaN,NaN,0.552975,0.188669
36663,work kamper,great way to adjust to rv living while earning...,it wa a physical job with demanding overtime s...,keep up the good work,work kamper great way to adjust to rv living w...,0.091667,0.011201,0.003498,0.003958,0.007364,0.000000,0.0000,0.750000,0.800000,0.142857,0.000000,0.600000,0.700000,0.560714,0.575000
36664,world class employer but only if you work at h...,smart colleague and management get to work on...,lack of job security for employee who work in ...,be more thoughtful about hiring decision,world class employer but only if you work at h...,0.475000,0.017922,0.021291,0.007377,0.024355,1.000000,0.0000,0.647619,0.304762,0.138750,-0.096250,0.500000,0.450000,0.449786,0.142929


In [75]:
extractor = PredictionFeaturesExtractor()

In [77]:
X_train_copy_final = extractor.fit_transform(X_train_copy_scaled, y_train_copy)

AttributeError: 'NoneType' object has no attribute 'transform'

In [119]:
X_train_copy_scaled.dropna(inplace=True)

In [120]:
X_train_copy_nb = add_multinomial_nb_prediction_feature(X_train_copy_scaled, y_train_copy)

ValueError: Found input variables with inconsistent numbers of samples: [20929, 36666]

# Train model

In [80]:
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
def train_RanForest(X_train, y_train, X_test, y_test, SCORE_COLS):
    prediction_scores_dict = {}
    for target in tqdm(SCORE_COLS):
        model = LogisticRegression(max_iter=1000)
        model.fit(X_train, y_train[target])
        prediction_scores[target] = model.score(X_test, y_test[target])

    return model, prediction_scores_dict